In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import time
import plotly.graph_objs as go
import plotly.tools as tls
import warnings
from collections import Counter
from sklearn.feature_selection import mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
test_df = pd.read_csv('/Users/macbookproi7/Desktop/PHYS 590/590/homework/homework5/test_data.csv',index_col=["id"])
train_df = pd.read_csv('/Users/macbookproi7/Desktop/PHYS 590/590/homework/homework5/train_data.csv',index_col=["id"])

In [3]:
test_df.fillna(-1,inplace=True)
train_df.fillna(-1,inplace=True) ###what to fill null values with
train_dummies= pd.get_dummies(train_df.select_dtypes(include=['O']))
test_dummies= pd.get_dummies(test_df.select_dtypes(include=['O']))

train = pd.concat([train_df.select_dtypes(include=['float64']),train_dummies],axis=1)
test=pd.concat([test_df.select_dtypes(include=['float64']), test_dummies],axis=1)


In [4]:
## We split the data into train and test
from sklearn.model_selection import train_test_split
train_dummies= pd.get_dummies(train_df)
y = train["target"]
X =  train.drop("target", 1)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.70,
                                                    random_state=69,
                                                    stratify = y)

In [5]:
# We fit and we predict
clf = LogisticRegression(n_jobs=-1, C=1e6)
clf.fit(X_train, y_train)

LogisticRegression(C=1000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [6]:
prediction = pd.DataFrame({"y": y_test, "prediction": clf.predict(X_test)})
(prediction["y"] == prediction["prediction"]).mean()


0.67023319615912214

In [7]:
from sklearn import metrics
metrics.accuracy_score(prediction["y"], prediction["prediction"])

0.67023319615912214

In [8]:
## Lets try different classifiers

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

lr_clf = LogisticRegression(n_jobs=-1)
tree_clf = DecisionTreeClassifier(min_samples_leaf=50)
knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance', n_jobs=-1)
gbm_clf = GradientBoostingClassifier()

In [9]:
lr_clf.fit(X_train, y_train)
tree_clf.fit(X_train, y_train)
knn_clf.fit(X_train, y_train)
gbm_clf.fit(X_train, y_train)

pred_df = pd.DataFrame({"y": y_test,
                        "pred_LR": lr_clf.predict(X_test),
                        "pred_KNN": knn_clf.predict(X_test),
                        "pred_Tree": tree_clf.predict(X_test),
                        "pred_GBM": gbm_clf.predict(X_test)})

In [10]:
from sklearn.linear_model import LogisticRegressionCV

lrcv_clf = LogisticRegressionCV(scoring="roc_auc", cv=10, Cs=100, n_jobs=-1)

lrcv_clf.fit(X_train, y_train)

LogisticRegressionCV(Cs=100, class_weight=None, cv=10, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
           refit=True, scoring='roc_auc', solver='lbfgs', tol=0.0001,
           verbose=0)

In [11]:
y_estimated_test = lrcv_clf.predict_proba(X_test)[:,1]
metrics.roc_auc_score(y_test, y_estimated_test)

0.62322274720994431

In [12]:
#from sklearn.model_selection import GridSearchCV
#from sklearn.model_selection import RandomizedSearchCV

#knn_clf = KNeighborsClassifier()

In [13]:
# param_grid = {
#     "n_neighbors": range(1,6),
#     "weights": ['uniform', 'distance'],
#     "p": [1,2],   
# }

# grid = GridSearchCV(knn_clf, param_grid=param_grid, scoring="roc_auc", n_jobs=-1, cv=3, verbose=1)

# grid.fit(X_train, y_train)

In [14]:
#y_estimated_test = grid.predict_proba(X_test)[:,1]
#metrics.roc_auc_score(y_test, y_estimated_test)

In [15]:
from scipy import stats
from sklearn.model_selection import RandomizedSearchCV 

param_distributions = {
    "learning_rate": stats.uniform(0.0001, 0.5),
    "n_estimators": stats.randint(100, 500),
    "max_depth": stats.randint(2, 20),
    "min_samples_split": stats.uniform(0, 1),
    "min_samples_leaf": stats.uniform(0, 0.5),
    "min_weight_fraction_leaf": stats.uniform(0, 0.5),
    "subsample":stats.uniform(0.2, 0.8),
    "max_leaf_nodes": stats.randint(4, 1000)    
}

gbm_clf = GradientBoostingClassifier()

r_grid = RandomizedSearchCV(gbm_clf, 
                            param_distributions=param_distributions,
                            cv=3,
                            n_jobs=-1,
                            n_iter=100,
                            verbose=1)

r_grid.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 14.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 20.8min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x111344e48>, 'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10ef7beb8>, 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10ef7bc18>, 'min_samples_split': <scip...0x10ef96780>, 'max_leaf_nodes': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10ef96908>},
          pre_dispatch='2*n_j

In [16]:
y_estimated_test = r_grid.predict_proba(X_test)[:,1]
metrics.roc_auc_score(y_test, y_estimated_test)


0.62274416162847801

In [17]:


submission_example = pd.DataFrame({"prediction": r_grid.predict_proba(X_test)[:, 1]}, index=X_test.index)


submission_example.to_csv("./submission_example.csv")